### Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import numpy as np

###  dataset path

In [2]:
root_dir = "../data"  # Change this to your dataset folder

# Load dataset (Train & Validation)
batch_size = 32
img_size = (224, 224)

train_dataset = image_dataset_from_directory(
    root_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_dataset = image_dataset_from_directory(
    root_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


Found 5945 files belonging to 40 classes.
Using 4756 files for training.
Found 5945 files belonging to 40 classes.
Using 1189 files for validation.


### Class names

In [3]:
class_names = train_dataset.class_names
num_classes = len(class_names)
print(f"Classes: {class_names}, Total Classes: {num_classes}")

# Normalize and preprocess images
def preprocess(image, label):
    image = preprocess_input(image)  # Apply EfficientNet preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


Classes: ['Aloevera', 'Amla', 'Amruta_Balli', 'Arali', 'Ashoka', 'Ashwagandha', 'Avacado', 'Bamboo', 'Basale', 'Betel', 'Betel_Nut', 'Brahmi', 'Castor', 'Curry_Leaf', 'Doddapatre', 'Ekka', 'Ganike', 'Gauva', 'Geranium', 'Henna', 'Hibiscus', 'Honge', 'Insulin', 'Jasmine', 'Lemon', 'Lemon_grass', 'Mango', 'Mint', 'Nagadali', 'Neem', 'Nithyapushpa', 'Nooni', 'Pappaya', 'Pepper', 'Pomegranate', 'Raktachandini', 'Rose', 'Sapota', 'Tulasi', 'Wood_sorel'], Total Classes: 40


### Load pre-trained EfficientNetB0

In [2]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(num_classes, activation="softmax")(x)  # Multi-class classification

# Create final model
model = Model(inputs=base_model.input, outputs=x)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Summary
model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][0]'] 

### Train the model

In [3]:
epochs = 2  # Adjust as needed
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)


Epoch 1/2


149/149 [==============================] - 159s 947ms/step - loss: 1.5037 - accuracy: 0.6491 - val_loss: 0.5192 - val_accuracy: 0.8873
Epoch 2/2
149/149 [==============================] - 139s 934ms/step - loss: 0.3559 - accuracy: 0.9220 - val_loss: 0.2752 - val_accuracy: 0.9369


### Load test dataset 

In [4]:
test_dataset = image_dataset_from_directory(
    root_dir,
    image_size=img_size,
    batch_size=batch_size
)

# Preprocess test dataset
test_dataset = test_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Evaluate model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")


Found 5945 files belonging to 40 classes.
186/186 [==============================] - 163s 873ms/step - loss: 0.1850 - accuracy: 0.9685
Test Accuracy: 0.9685


### Get true labels and predictions and classification report

In [5]:
y_true, y_pred = [], []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())
    
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))


1/1 [==============================] - 3s 3s/step
               precision    recall  f1-score   support

     Aloevera     0.9760    0.9939    0.9849       164
         Amla     0.9667    0.9932    0.9797       146
 Amruta_Balli     0.9424    0.8973    0.9193       146
        Arali     0.9932    0.9932    0.9932       146
       Ashoka     1.0000    0.9863    0.9931       146
  Ashwagandha     0.9732    0.9932    0.9831       146
      Avacado     0.9728    0.9795    0.9761       146
       Bamboo     0.9932    1.0000    0.9966       146
       Basale     0.9333    0.9589    0.9459       146
        Betel     0.9861    0.9404    0.9627       151
    Betel_Nut     1.0000    1.0000    1.0000       146
       Brahmi     0.9730    0.9863    0.9796       146
       Castor     0.9876    0.9938    0.9907       160
   Curry_Leaf     0.9928    0.9452    0.9684       146
   Doddapatre     0.9799    1.0000    0.9898       146
         Ekka     0.9660    0.9726    0.9693       146
       Ganike 

### Model saveing

In [7]:
model.save("../models/EfficientNetModel.h5")
print("Model saved as 'EfficientNetModel.h5'")


C:\Users\dell\anaconda3\envs\multi\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as 'EfficientNetModel.h5'
